In [31]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import imutils
from tensorflow import keras

In [32]:
bg = None

In [33]:
model = keras.models.load_model("TrainedModel/honeyGestureRecogModel.tfl")

In [34]:
def resizeImage(imageName):
    basewidth = 100
    img = Image.open(imageName)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    img.save(imageName)

In [35]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [36]:
def segment(image, threshold=25):
    
    #1  -- > get diff between forground and backgorund
    #2  ---> assign white pixels if the diff is greater than threshold
    #3  --> extract those white pizels using contours
    
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground ( if the pixel val > thre, assign 1 else 0 , so you get pixel val as white where obj is present)
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    #finding contours is like finding white object from black background
    (cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [37]:
def main():
    
    #background subraction algorithm
    # -- get a estimate of the background then get the current image frame and subtract the estimated backgorund from it to get only the segments important ( foreground)
    #estimated background is nothing but the previous image and in this case running avergae of pevious images
    
    
    
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0
    start_recording = False

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width = 700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                if start_recording:
                    cv2.imwrite('Temp.png', thresholded)
                    resizeImage('Temp.png')
                    predictedClass, confidence = getPredictedClass()
                    showStatistics(predictedClass, confidence)
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
        
        if keypress == ord("s"):
            start_recording = True


In [51]:
def getPredictedClass():
    # Predict
    image = cv2.imread('Temp.png')
    print("image shape = ", image.shape)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print("gray_image.shape = ", gray_image.shape)
    #gray_image.resize((89,100))
    print("gray_image rehspae shape = ", gray_image.reshape(89, 100, 1).shape)
    prediction = model.predict([gray_image.reshape(-1,89, 100, 1)])
    return np.argmax(prediction), (np.amax(prediction) / (prediction[0][0] + prediction[0][1] + prediction[0][2]))

In [52]:
def showStatistics(predictedClass, confidence):

    textImage = np.zeros((300,512,3), np.uint8)
    className = ""

    if predictedClass == 0:
        className = "fist"
    elif predictedClass == 1:
        className = "hi"
    elif predictedClass == 2:
        className = "peace"

    cv2.putText(textImage,"Pedicted Class : " + className, 
    (30, 30), 
    cv2.FONT_HERSHEY_SIMPLEX, 
    1,
    (255, 255, 255),
    2)

    cv2.putText(textImage,"Confidence : " + str(confidence * 100) + '%', 
    (30, 100), 
    cv2.FONT_HERSHEY_SIMPLEX, 
    1,
    (255, 255, 255),
    2)
    cv2.imshow("Statistics", textImage)

In [54]:
main()

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap

gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap

...

gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image

gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap

image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shape =  (89, 100, 3)
gray_image.shape =  (89, 100)
gray_image rehspae shape =  (89, 100, 1)
image shap